# Sistem za klasifikaciju email-ova

## Uvod

Ovaj notebook prikazuje dva pristupa klasifikaciji e-mail poruka na **spam** i **ham**:

1. **Klasični pristup (Bag-of-Words + Naive Bayes)**
   - Čišćenje i normalizacija teksta
   - Numerička reprezentacija pomoću BoW
   - Trening jednostavnog Naive Bayes modela
   - Evaluacija modela: Accuracy, Precision, Recall, F1-score, Precision Matrix
   - Opcionalno balansiranje klasa sa SMOTE

2. **Napredni pristup (BERT Transformer)**
   - Korišćenje unapred treniranog BERT modela (DistilBERT-a. RoBERTa, BERT-Base)
   - Fine-tuning BERT-a na našem dataset-u za klasifikaciju spam/ham
   - Evaluacija performansi na test skupu
   - Upoređivanje rezultata sa Naive Bayes klasifikatorom

**Cilj:** Demonstrirati razliku između klasičnog modela i moćnog modernog NLP transformera, kao i razumeti prednosti BERT-a kod tekstualnih klasifikacija.

## Klasifikaciju pomoću Naive Bayes-a

### Biblioteke

Uvoz potrebnih biblioteka za obradu podataka, metrike, balansiranje klasa i vizualizaciju.

In [ ]:
import pandas as pd
import re
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot as plt

### Funkcije za čišćenje code-a

Normalizacija teksta: mala slova, uklanjanje linkova, brojeva i specijalnih karaktera, višestrukih razmaka.

In [ ]:
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"subject:", "", text)
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"[^a-z\s]", " ", text)
    text = re.sub(r"\s+", " ", text)

    return text.strip()

: 

## Učitavanje dataset-a

-Učitavanje Enron i Venky dataset-a

-Kreiranje full_text kolone i numeričkih labela

In [ ]:
def load_datasets():
    df_enron = pd.read_csv("../data/enron_mails.csv").dropna(subset=["Message"])
    df_venky = pd.read_csv("../data/venky_spam_ham_dataset.csv").dropna(subset=["text"])

    df_enron["full_text"] = (
        df_enron["Subject"].fillna("") + " " + df_enron["Message"].fillna("")
    ).apply(clean_text)
    df_venky["text"] = df_venky["text"].apply(clean_text)

    df_enron["label_num"] = df_enron["Spam/Ham"].map({"ham": 0, "spam": 1})
    df_venky["label_num"] = df_venky["label"].map({"ham": 0, "spam": 1})

    return df_enron, df_venky

#primer prikaza nekoliko redova
df_enron, df_venky = load_datasets()
df_enron.head()